# Demonstrate Interaction of pydantic and omnikeeper

In [12]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER, get_all_traitentities_pydantic
import uuid
from pydantic import BaseModel,TypeAdapter, PlainSerializer
from typing_extensions import Annotated
from typing import List

In [13]:
trait_id = "python_client_demo.test"
layer_id = "testlayer"

create a omnikeeper client

In [14]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

ensure trait exists

In [15]:
ret = okc.upsert_trait(okapiclient, TraitDefinition(trait_id, [
        TraitAttributeDefinition("id", "test.id", ATTRIBUTETYPE_INTEGER),
        TraitAttributeDefinition("array", "test.array", ATTRIBUTETYPE_TEXT, is_array=True),
        TraitAttributeDefinition("some_key", "test.mapped_some_key", ATTRIBUTETYPE_TEXT),
      ]))
assert(ret)

ensure layer exists

In [16]:
ret = okc.create_layer(okapiclient, layer_id, "just a test layer", okc.hex_string_to_rgb_color("#6666FF"))
assert(ret)

Define pydantic class, same as trait, with the addition of a ciid field

In [17]:
class Test(BaseModel):
    ciid: okc.SerializableUUID # this is the short form for an annotated UUID field that is properly serializable to a string
    id: int
    array: List[str] = ['foo', 'bar'],
    some_key: str

Init data

In [18]:
data_init = [
    Test(ciid=uuid.uuid4(), id=1, array=["a", "b"], some_key="Value 1"),
    Test(ciid=uuid.uuid4(), id=3, array=["c", "d"], some_key="another Value"),
]
data_init

[Test(ciid=UUID('28f37810-e012-4883-8a03-e97ab6c8b344'), id=1, array=['a', 'b'], some_key='Value 1'),
 Test(ciid=UUID('aefd377f-fd95-4061-8e5b-b8068e8a2b74'), id=3, array=['c', 'd'], some_key='another Value')]

Write data to omnikeeper

In [19]:
ret = okc.bulk_replace_trait_entities_pydantic(okapiclient, trait_name=trait_id, input=data_init, write_layer=layer_id)
assert(ret)

Read data back from omnikeeper, should be the same

In [20]:
m = get_all_traitentities_pydantic(okapiclient, trait_name=trait_id, ta=TypeAdapter(List[Test]), layers=[layer_id])
m

[Test(ciid=UUID('28f37810-e012-4883-8a03-e97ab6c8b344'), id=1, array=['a', 'b'], some_key='Value 1'),
 Test(ciid=UUID('aefd377f-fd95-4061-8e5b-b8068e8a2b74'), id=3, array=['c', 'd'], some_key='another Value')]

Change some data locally

In [21]:
m[0].array=['x', 'y']

Write it to omnikeeper again and read it again

In [22]:
ret = okc.bulk_replace_trait_entities_pydantic(okapiclient, trait_name=trait_id, input=m, write_layer=layer_id)
assert(ret)
m = get_all_traitentities_pydantic(okapiclient, trait_name=trait_id, ta=TypeAdapter(List[Test]), layers=[layer_id])
m

[Test(ciid=UUID('28f37810-e012-4883-8a03-e97ab6c8b344'), id=1, array=['x', 'y'], some_key='Value 1'),
 Test(ciid=UUID('aefd377f-fd95-4061-8e5b-b8068e8a2b74'), id=3, array=['c', 'd'], some_key='another Value')]